# Users and Items Transactions

In [1]:
import numpy as np
import pandas as pd
from time import time
from numpy import save, load

In [2]:
tic = time()
TRIGGER = 'M'

if TRIGGER == 'K':
    file_name = '../datasets/file_users_100K.txt'
#     file_name = '../datasets/file_items_100K.txt'

    data = load('../datasets/main/train_set_100K.npy')
#     data = np.loadtxt('../datasets/main/ml-100k.data', skiprows=0, delimiter='\t').astype('int32')

elif TRIGGER == 'M':
    file_name = '../datasets/file_users_1M.txt'
#     file_name = '../datasets/file_items_1M.txt'

    data = load('../datasets/main/train_set_1M.npy')
#     data = np.loadtxt('../datasets/main/ml-1M.dat', skiprows=0, delimiter='::').astype('int32')

In [3]:
data

array([[        1,         1,         5, 978824268],
       [        1,        48,         5, 978824351],
       [        1,       150,         5, 978301777],
       ...,
       [     6040,      3703,         4, 964828575],
       [     6040,      3751,         4, 964828782],
       [     6040,      3819,         5, 963272166]])

In [4]:
data_csv = pd.DataFrame(data, columns = ['user', 'item', 'rating', 'timestamp'])
data_csv.sort_values(['user', 'item'], ascending=[True, True], inplace=True)
data_csv.reset_index(inplace=True)
data_csv.drop('index', axis=1, inplace=True)
data_csv

,user,item,rating,timestamp
0,1,1,5,978824268
1,1,48,5,978824351
2,1,150,5,978301777
3,1,260,4,978300760
4,1,527,5,978824195
...,...,...,...,...
900184,6040,3671,4,997454367
900185,6040,3683,4,960971696
900186,6040,3703,4,964828575
900187,6040,3751,4,964828782


In [5]:
users = np.unique(data[:, 0]).tolist() # list of unique users
items = np.unique(data[:, 1]).tolist() # list of unique items

In [6]:
n_u = len(users)     # number of users
n_m = len(items)     # number of movies
n_r = data.shape[0]  # number of ratings

print("USERS: {}\t ITEMS: {}\t RATINGS: {}".format(n_u, n_m, n_r))

USERS: 6040	 ITEMS: 3694	 RATINGS: 900189


In [7]:
# Make dict for users and movies, where udict[u_id] = index(0..942), and mdict[m_id] = index(0..1681)
udict = {}
for i, u_id in enumerate(users):
    udict[u_id] = i
    
mdict = {}
for i, m_id in enumerate(items):
    mdict[m_id] = i

In [8]:
# Empty matrix
matrix = np.zeros((n_u, n_m), dtype='int32')

In [9]:
for i in range(n_r):
    u_id = data[i, 0]
    m_id = data[i, 1]
    r = data[i, 2]
    
    matrix[udict[u_id], mdict[m_id]] = int(r)

In [10]:
matrix

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]])

## Save Dataset for Apriory

#### Users Transactions

In [11]:
records = []

for u_id in users:
    temp = []
    
    for i_id in items:
        r = matrix[udict[u_id], mdict[i_id]]
        if r != 0:
            temp.append('Item_{0}: {1}'.format(i_id, r))
            
    records.append(temp)

#### Items Transactions

In [12]:
# records = []

# for i_id in items:
#     temp = []
    
#     for u_id in users:
#         r = matrix[udict[u_id], mdict[i_id]]
#         if r != 0:
#             temp.append('User_{0}: {1}'.format(u_id, r))
            
#     records.append(temp)

In [13]:
with open(file_name, "w") as file:
    for line in records:
        if line != records[0]:
            file.write('\n')
        
        for item in line:
            if item != line[-1]:
                file.write(item + ', ')
            else:
                file.write(item)
    file.close()

In [14]:
print('Preprocessing time: {} seconds!'.format(int(time() - tic)))

Preprocessing time: 20 seconds!
